In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

In [0]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D,GlobalAveragePooling2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [0]:
number_train = len(os.listdir("../input/train"))
train_df = pd.read_csv("../input/train.csv")
print('The shape of trainning csv is ' + str(train_df.shape))
print('The number of trainning image is ' + str(number_train))
train_df.head()

In [0]:
class_size = train_df['Id'].value_counts()
class_size[1:15].plot(kind='bar')

In [0]:
def prepareImages(data, m, dataset):
    print("Preparing images")
    X_train = np.zeros((m, 100, 100, 3))
    count = 0
    
    for fig in data['Image']:
        #load images into images of size 100x100x3
        img = image.load_img("../input/"+dataset+"/"+fig, target_size=(100, 100, 3)) # show the image as PIL  
        x = image.img_to_array(img) # convert image to pixal values and shapes as target_size which defined above 
        x = preprocess_input(x) # center the image pixal value because Resnet model train on pixal values in (-1,1)

        X_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [0]:
X = prepareImages(train_df, train_df.shape[0], "train")
X /= 255 # normalized the image pixal values  
X.shape # just check the above model 

In [0]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded) this change 5005 classes as lable class as a np.array

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1) # change integer_encoded to (25361, 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded # change all sample as (25361, 5005) and  each row is one sample one-encoding class 
    # print(y.shape)
    return y, label_encoder

In [0]:
# Another way to get lable in one-hot encoding 
yy = pd.get_dummies(train_df['Id']).values # try another way to change one-encoding 

In [0]:
# try to split the train and val
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X, yy, test_size = 0.2, random_state = 0)

In [0]:
# check the train and val
print(train_x.shape, val_x.shape)
print(train_y.shape, val_y.shape)

In [0]:
y, label_encoder = prepare_labels(train_df['Id'])
y.shape

In [0]:
# The first CNN model 
model = Sequential()

model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

model.add(BatchNormalization(axis = 3, name = 'bn0'))
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2), name='max_pool'))
model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3), name='avg_pool'))

model.add(Flatten())
model.add(Dense(500, activation="relu", name='rl'))
model.add(Dropout(0.5))
model.add(Dense(yy.shape[1], activation='softmax', name='sm'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

In [0]:
# Try another CNN model 
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (100,100,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
model.add(Dropout(0.25))

# fully connected
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(yy.shape[1], activation = "softmax"))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

In [0]:
history = model.fit(X, yy, epochs=1, batch_size=100, verbose=1)

In [0]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [0]:
# try fit_generator funtion for data augmentation avoiding overfiting
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
history_1 = model.fit_generator(train_datagen.flow(train_x, train_y, batch_size=32),
                              steps_per_epoch=len(train_x) / 32,
                              epochs=10,
                              validation_data=validation_datagen.flow(val_x, val_y, batch_size=32),
                              validation_steps=len(val_x) / 32)


In [0]:
plt.plot(history_1.history['acc'])
plt.plot(history_1.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [0]:
# try transfer learning Inceptionv3
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model

pre_trained_model = InceptionV3(input_shape = (100, 100, 3), 
                                include_top = False,weights=None, classes=5005)

#pre_trained_model.load_weights(local_weights_file)

#for layer in pre_trained_model.layers:
#  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [0]:
x = Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = Dense(5005, activation='softmax')(x)           

model_1 = Model(pre_trained_model.input, x) 

In [0]:
# try transfer learning ResNet50
from keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model

pre_trained_model_1 = ResNet50(input_shape = (100, 100, 3), 
                                include_top = False,weights=None, classes=5005)

x = pre_trained_model_1.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

x = Dense(5005, activation='softmax')(x)

model_2 = Model(inputs=pre_trained_model_1.input, outputs=x)

model_2.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [0]:
pre_trained_model_1.summary()

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen_1 = ImageDataGenerator(rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen_1 = ImageDataGenerator()

history_2 = pre_trained_model_1.fit_generator(train_datagen_1.flow(train_x, train_y, batch_size=32),
                              steps_per_epoch=len(train_x) / 32,
                              epochs=10,
                              validation_data=validation_datagen_1.flow(val_x, val_y, batch_size=32),
                              validation_steps=len(val_x) / 32)X

In [0]:
# Try MobileNet transfer learning 
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
model_3 = MobileNet(input_shape=(100, 100, 3), alpha=1., weights=None, classes=5005)

model_3.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen_2 = ImageDataGenerator(rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen_2 = ImageDataGenerator()

history_3 =model_3.fit_generator(train_datagen_2.flow(X, yy, batch_size=32),
                              steps_per_epoch=len(X) / 32,
                              epochs=10)